### Алгоритмы и лямбды

https://en.cppreference.com/w/cpp/algorithm

https://en.cppreference.com/w/cpp/numeric

https://en.cppreference.com/w/cpp/language/lambda

[C++ Lambda Idioms - Timur Doumler - CppCon 2022](https://www.youtube.com/watch?v=xBAduq0RGes)

<br />

##### Примеры и философия алгоритмов

`accumulate` для знакомства

```c++
int sum_98(const std::vector<int>& v)
{
    int rv = 0;
    for (std::vector<int>::const_iterator it = v.begin(), e = v.end(); it != e; ++it)
        rv += *it;
    return rv;
}

int sum_range_for_11(const std::vector<int>& v)
{
    int rv = 0;
    for (int x : v)
        rv += x;
    return rv;
}

int sum_stl_accumulate(const std::vector<int>& v)
{
    // https://en.cppreference.com/w/cpp/algorithm/accumulate
    return std::accumulate(v.begin(), v.end(), 0);
    
    // как это читать:
    // std::accumulate(  // просуммировать элементы из последовательности (std::plus - дефолтная операция у accumulate)
    //      v.begin(),   // итератор на первый элемент
    //      v.end(),     // итератор на следующий за последним
    //      0);          // начальное значение суммы
}

int sum_stl_reduce(const std::vector<int>& v)
{
    // https://en.cppreference.com/w/cpp/algorithm/reduce (C++17)
    return std::reduce(v.begin(), v.end());
}
```

Референсная реализация `std::accumulate` (см. документацию):

```c++
template<typename InputIt,
         typename T,
         typename BinaryOperation>
T accumulate(InputIt first,
             InputIt last,
             T init, 
             BinaryOperation op)
{
    for (; first != last; ++first)
        init = op(init, *first);

    return init;
}
```

<br />

`count` - ещё пример

```c++
size_t count_range_for_11(const std::vector<int>& v, int val)
{
    size_t rv = 0;
    for (auto x : v)
        if (x == val)
            ++rv;
    return rv;
}

size_t count_stl(const std::vector<int>& v, int val)
{
    return std::count(v.begin(), v.end(), val);
}
```

Референсная реализация `std::count`:

```c++
template<typename InputIt,
         typename T>
typename iterator_traits<InputIt>::difference_type
count(InputIt first,
      InputIt last,
      const T& value)
{
    typename iterator_traits<InputIt>::difference_type ret = 0;
    for (; first != last; ++first) {
        if (*first == value)
            ret++;
    }
    return ret;
}
```

<br />

`find`

```c++
bool contains_range_for_11(const std::vector<int>& v, int val)
{
    for (int x : v)
        if (x == val)
            return true;
    return false;
}

bool contains_stl(const std::vector<int>& v, int val)
{
    return std::find(v.begin(), v.end(), val) != v.end();
}
```

На примере `find` обсудить вопрос что лучше: наивная реализация или переиспользование алгоритма:
* какой код чище?
* вопрос уровня абстракции
* оптимизации внутри алгоритмов
* проблема читабельности алгоритмов:
    * сколько служебных конструкций и повторений содержит элементарный код проверки, что `v` содержит `val`?
    
    ```c++
    std::find(
        current_students_in_the_class.begin(),
        current_students_in_the_class.end(),
        "those little boy Einstein")
        != current_students_in_the_class.end();
    ```

* ranges to the rescue! (но не совсем и с кучей оговорок)

<br />

##### Алгоритмы работают с итераторами!

Теперь мы готовы осознать, что итератор - не про итерирование.

Итератор для алгоритмов требует реализацию методов, но КАК они будут реализованы - вопрос.


```c++
template<typename T>
class MyIterator {
    T& operator *() const {
        // какая-то реализация
    }
    
    void operator++() const {
        // какая-то реализация, возможно, пустая
    }
    
    ...    
    
};
```

Примеры:
* `std::ostream_iterator<int>(std::cout, " ")` - ~итератор, который выводит записываемое значение в `std::cout` через пробел
* `std::back_inserter(v)` - ~итератор, который делает `push_back` записываемых значений в вектор `v`

**Пример**: разные варианты сортировки через один алгоритм

```c++
// отсортируем вектор по возрастанию
void mysort_ascending(std::vector<int>& v)
{
    std::sort(v.begin(), v.end());
}

// отсортируем вектор по убыванию
void mysort_descending(std::vector<int>& v)
{
    std::sort(v.rbegin(), v.rend());
}
```

**Пример**: вывод на консоль через алгоритм (подробно объяснить пример)

```c++
// copy - копирует последовательность
//        https://en.cppreference.com/w/cpp/algorithm/copy
void copy_example()
{
    std::vector<int> v = {1, 2, 3};
    std::list<int> l = {4, 5, 6};
    
    std::copy(v.begin(), v.end(), l.begin());
}

// copy - копирует последовательность в OutputIt,
//        но ведь можно и подшаманить OutputIt
void print_sequence()
{
    std::vector<int> v = {1, 2, 3, 4, 5};
    
    std::copy(v.begin(), v.end(),
              std::ostream_iterator<int>(std::cout, " "));
}
```

**Пример**: чтение (подробно объяснить пример)

```c++
std::vector<int> read_ints()
{
    return std::vector<int>{
        std::istream_iterator<int>(std::cin),
        std::istream_iterator<int>()
    };
}

int sum_ints()
{
    std::istringstream iss("1 2 3 4 5");
    
    return std::reduce(
        std::istream_iterator<int>(iss),
        std::istream_iterator<int>());
}
```

<br />

И в то же время гибкости класса итераторов не хватает, чтобы простые конструкции реализовывались без лишних раздумий:

```c++
void copy_example_bug()
{
    std::vector<int> a = {1, 2, 3, 4, 5};
    std::vector<int> b;

    std::copy(a.begin(), a.end(), b.begin());
}

void copy_example_fix1()
{
    std::vector<int> a = {1, 2, 3, 4, 5};
    std::vector<int> b;

    b.resize(a.size(), 0);
    std::copy(a.begin(), a.end(), b.begin());
}

void copy_example_fix2()
{
    std::vector<int> a = {1, 2, 3, 4, 5};
    std::vector<int> b;

    std::copy(a.begin(), a.end(), std::back_inserter(b));
    // back_inserter - специальный итератор, который в operator = вызывает b.push_back(...)
}
```

<br />

```c++
void replace_copy_bug()
{
    std::vector<int> a = {1, 2, 3, 4, 5, 1, 2, 3, 4, 5};
    std::vector<int> b;

    std::replace_copy(a.begin(), a.end(), b.begin(), 2, 7);
}

void replace_copy_fix()
{
    std::vector<int> a = {1, 2, 3, 4, 5, 1, 2, 3, 4, 5};
    std::vector<int> b;

    std::replace_copy(a.begin(), a.end(), std::back_inserter(b), 2, 7);
}

void replace_copy_better()
{
    std::vector<int> a = {1, 2, 3, 4, 5, 1, 2, 3, 4, 5};
    std::vector<int> b;

    b.reserve(a.size());
    std::replace_copy(a.begin(), a.end(), std::back_inserter(b), 2, 7);
}
```

<br />

##### `std::remove` и `std::unique` - где чаще всего ошибаются

```c++
void remove_usage_bug()
{
    std::vector<int> a = {1, 2, 3, 4, 5, 1, 2, 3, 4, 5};
    std::remove(a.begin(), a.end(), 2);
}

void remove_usage_fix()
{
    std::vector<int> a = {1, 2, 3, 4, 5, 1, 2, 3, 4, 5};
    auto new_end = std::remove(a.begin(), a.end(), 2);
    a.resize(std::distance(a.begin(), new_end));
}

void remove_usage_fix_another_option()
{
    std::vector<int> a = {1, 2, 3, 4, 5, 1, 2, 3, 4, 5};
    a.erase(std::remove(a.begin(), a.end(), 2),
            a.end());
}

void remove_usage_list()
{
    std::list<int> l = {1, 2, 3, 4, 5, 1, 2, 3, 4, 5};
    l.remove(2); // list is an exception!
}
```

```c++
void unique_usage_bug()
{
    std::vector<int> a = {1, 2, 3, 4, 5, 1, 2, 3, 4, 5};
    std::sort(a.begin(), a.end());  // unique removes only adjacent uniques
    std::unique(a.begin(), a.end());
}

void unique_usage_fix()
{
    std::vector<int> a = {1, 2, 3, 4, 5, 1, 2, 3, 4, 5};
    std::sort(a.begin(), a.end()); // unique removes only adjacent uniques
    auto new_end = std::unique(a.begin(), a.end());
    a.resize(std::distance(a.begin(), new_end));
}

void unique_usage_list()
{
    std::list<int> l = {1, 2, 3, 4, 5, 1, 2, 3, 4, 5};
    l.sort();  // unique removes only adjacent uniques
    l.unique();
}
```

<br />

Обратить внимание на состояние контейнера после вызова remove:

```c++
void remove_result_content()
{
    std::vector<int> a = {1, 2, 3, 4, 5, 1, 2, 3, 4, 5};
    auto new_end = std::remove(a.begin(), a.end(), 2);
    
    for (int x : a)
        std::cout << x << " "; // ???
}
```

<br />

######  `std::erase` (C++ 20)

https://en.cppreference.com/w/cpp/container/vector/erase2

Для проблемы вызова `std::remove` по контейнерам решили написать shortcut для `std::vector`:

(закинуть на godbolt на последний gcc, не забыть `-std=c++20`)

```c++
#include <algorithm>
#include <iostream>
#include <iterator>
#include <string>
#include <vector>

int main() {
    std::vector<std::string> container = {
        "Carthago", "Rome", "Rome",
        "Carthago", "Paris", "Ciaro",
        "Konstantinie"};
    std::erase(container, "Carthago");

    std::copy(
        container.begin(),
        container.end(),
        std::ostream_iterator<std::string>(std::cout, " "));
    return 0;
}
```

Вывод ожидаем:
    
```
Rome Rome Paris Ciaro Konstantinie
```

<br />

Но если так (см. на godbolt):

```c++
    ...
    std::vector<std::string> container = {
        "Carthago", "Rome", "Rome",
        "Carthago", "Paris", "Ciaro",
        "Konstantinie"};
    std::erase(container, container.front());
    ...
```

то вывод будет другим:

```
Rome Carthago Paris Ciaro Konstantinie
```

**Вопрос**: почему?

<details>
<summary>Ответ</summary>

<p>
    std::erase - шаблонный алгоритм. Он сравнивает по ссылке на первый элемент. std::erase модифицирует контейнер, вместе с ним модифицируя и первый элемент. Т.е. элемент для удаления изменяется неявно в процессе алгоритма.
</p>
</details>


**Вопрос**: как починить?

<details>
<summary>Ответ</summary>

<p>
    сделать копию первого элемента и сравнивать с ней :(
    
    std::erase(container, std::string(container.front()));    
    или    
    std::erase(container, auto{container.front()});  // since C++23
</p>
</details>

<br />

###### `std::sort`-like and comparator

https://en.cppreference.com/w/cpp/algorithm/sort  
https://en.cppreference.com/w/cpp/algorithm/stable_sort  
https://en.cppreference.com/w/cpp/algorithm/partial_sort  
https://en.cppreference.com/w/cpp/algorithm/nth_element  

Алгоритмы `std::sort`, `std::stable_sort` и другие имеют требование на Comparator - сравнение должно удовлетворять органичениям strict weak ordering (иначе - UB).

https://en.cppreference.com/w/cpp/named_req/Compare

* For all `a`, `comp(a, a) == false`.
* If `comp(a, b) == true` then `comp(b, a) == false`.
* If `comp(a, b) == true` and `comp(b, c) == true` then `comp(a, c) == true`.

Примеры часто втсречаемых ошибок в кодовой базе С++. Найдите их (ошибки):

```c++
//
// Example 1: Сортировка по возрастанию
//
std::vector<int> v = {5, 4, 3, 2, 1};
std::sort(v.begin(), v.end(), [](int l, int r){
    return l <= r;
});
```

```c++
//
// Example 2: уступаем место девочкам
//
struct Person {
    std::string name;
    Gender gender;
};

std::vector<Person> people = ...;
std::sort(people, [](const Person &lhs, const Person &rhs) {
    if (lhs.gender == Gender::Female)
        return true;
    if (rhs.gender == Gender::Female)
        return false;
    return lhs.name < rhs.name;
});
```

<br />

### Лямбды - использование

`find` по условию через функцию:

```c++
bool is_underage(const Person& p) {
    return p.age < 18;
}

std::vector<Person> people = { ... };

auto it = std::find_if(
    people.begin(),
    people.end(),
    is_underage);

if (it != people.end())
    std::cout << it->age << std::endl;
```

`find` по условию через лямбду:

```c++
std::vector<Person> people = { ... };

auto it = std::find_if(
    people.begin(),
    people.end(),
    [](const Person& p){ return p.age < 18; });
    
if (it != people.end())
    std::cout << it->age << std::endl;
```

<br />

`count_if` - подсчёт числа элементов

```c++
std::vector<Person> people = { ... };
        
const int underage_count = std::count_if(people.begin(), people.end(),
                                         [](const Person& p){ return p.age < 18; });
```

<br />

`all_of`, `any_of`, `none_of`

```c++
auto is_underage = [](const Person& p){ return p.age < 18; };

bool all_young = std::all_of(people.begin(), people.end(), is_underage);
bool any_young = std::any_of(people.begin(), people.end(), is_underage);
bool all_old   = std::none_of(people.begin(), people.end(), is_underage);
```

<br />

`transform`

```c++
void toupper_inplace(std::string& s)
{
    std::transform(
        s.begin(), s.end(),  // input range
        s.begin(),           // output
        [](unsigned char c) -> unsigned char { return std::toupper(c); });  // function
}

void print_toupper(std::string& s)
{
    std::tranform(
        ...  // Упражнение
    );
}

// map-reduce example
int sum_of_squarries(const std::vector<int>& v)
{
    return std::transform_reduce(
        v.begin(), v.end(),                     // последовательность
        0,                                      // начальный элемент в reduce-шаге (как в accumulate)
        [](int sum, int x) { return sum + x; }, // операция reduce
        [](int x){ return x * x; });            // операция transform
}
```

<br />

binary search:

```c++
std::vector<int> v = { 1, 1, 2, 3, 4, 4, 4, 5, 5, 6 };
        
auto lower = std::lower_bound(v.begin(), v.end(), 4);
auto upper = std::upper_bound(v.begin(), v.end(), 4);
        
std::copy(lower, upper,
          std::ostream_iterator<int>(std::cout, " "));
// output: 4 4 4
```

```c++
if (std::binary_search(v.begin(), v.end(), 5))
    std::cout << "found";
else
    std::cout << "not found";
```

<br />

`set` operations:

**Замечание**: специфика set operations, что они работают только с отсортированными последовательностями

```c++
std::vector<int> v1 = {...};
std::sort(v1.begin(), v1.end());
        
std::vector<int> v2 = {...};
std::sort(v2.begin(), v2.end());

// check for subset:
if (std::includes(v1.begin(), v1.end(),
                  v2.begin(), v2.end())) {
    std::cout << "v2 is subset of v1";
}

// build union
std::vector<int> v3 = {...};
std::set_union(v1.begin(), v1.end(),  // отсортированная последовательность 1
               v2.begin(), v2.end(),  // отсортированная последовательность 2
               std::back_inserter(v3));  // куда складывать результат
// что здесь можно было бы добавить?
```

<br />

`min`/`max`

```c++
std::vector<int> v = { ... };

auto it_min = std::min_element(v.begin(), v.end());
auto it_max = std::max_element(v.begin(), v.end());
// почему возвращается итератор, а не сразу ссылка на минимальный элемент?
```

```c++
std::vector<Person> people = { ... };
        
auto it_min = std::min_element(people.begin(), people.end(),
                               [](const Person& l, const Person& r) {
                                   return l.name < r.name;
                               });  // поиск первого по алфавиту

auto it_max = std::max_element(people.begin(), people.end(),
                               [](const Person& l, const Person& r) {
                                   return l.age < r.age;
                               });  // поиск самого старшего
```

<br />

etc. etc. etc.

<br />

За рамками рассказа:
* boost algorithms
* bit manipulations
* math operations
* другие реализации алгоритмов (EASTL)

<br />

### Лямбды - детали

<br />

##### Откуда ноги растут - из функторов

Во времена до С++11:

```c++
bool is_underage(const Person& person)
{
    return person.age < 18;
}

std::vector<Person> people = { ... };

auto it = std::find_if(people.begin(), people.end(), is_underage);
```

Либо через функтор:

```c++
struct IsUnderageChecker
{
    bool operator()(const Person& person) const
    {
        return person.age < 18;
    }    
};

std::vector<Person> people = { ... };
        
auto it = std::find_if(people.begin(), people.end(), IsUnderageChecker());
```

Функтор - класс с определённым `operator()`. Объекты такого класса можно "вызывать":

**Пример**:

```c++
Person ilya{"Ilya Muromec", 32};

IsUnderageChecker is_underage_checker;

if (is_underage_checker(ilya))
    std::cout << "Ilya is old";
else
    std::cout << "Ilya is young";
```

Конкретно для этого примера (с некоторыми оговорками) не важно, свободная функция `is_underage` или функтор `IsUnderageChecker`, но иногда может понадобится и класс.

**Упражение**: Приведите пример. Подсказка: в чём большое отличиие функции от класса?

**Вариант**:

```c++
class АgeLessChecker {
public:
    explicit АgeLessChecker(int bound) : bound_(bound) {}
    
    bool operator()(const Person& person) const {
        return person.age < bound_;
    }
    
private:
    int bound_;
};


Person ilya{"Ilya Muromec", 32};

АgeLessChecker ilya_maturity_checker{33};

if (ilya_maturity_checker(ilya))
    std::cout << "time to fight!";
```

**Вариант**:

```c++
class NameInSetChecker {
public:
    explicit NameInSetChecker(std::set<std::string> allowed_names)
        : allowed_names_(std::move(allowed_names))
    {}
    
    bool operator()(const Person& person) const {
        return allowed_names_.count(person.name) > 0;
    }
    
private:
    std::set<std::string> allowed_names_;
};

std::vector<Person> people = { ... };

const std::set<std::string> allowed_names({"Balin", "Dvalin", "Gloin"});

auto it = std::find_if(people.begin(), people.end(),
                       NameInSetChecker(allowed_names));
```

**Более интересный пример**:

```c++
struct HTMLPrinter
{
    void operator()(int x) const { std::cout << "<p> int value = " << x << " </p>"; }
    void operator()(double x) const { std::cout << "<p> dlb value = " << x << " </p>"; }
};

HTMLPrinter printer;
printer(5);    // <p> int value = 5 </p>
printer(5.0);  // <p> dbl value = 5.000000 </p>
```

[Правила работы с функторами](https://cpp.com.ru/meyers/ch6.html). Основное:
* Классы функторов для алгоритмов передаются по значению, а значит их следует проектировать копируемыми
* Желательно, чтобы функторы для алгоритмов не изменяли своё состояние (нет гарантий, что функтор не будет раскопирован))

<br />

*Функторы - гибкий и богатый инструмент, за исключением того, что требуют большого объёма кода для реализации. Лямбду можно рассматривать как сокращённую запись функтора, который компилятор сгенерирует неявно.*

<br />

##### Контекст, захват и передача параметров. Dangling references, правила


Лямбда без состояния:

```c++
auto it = std::find_if(people.begin(), people.end(),
                       [](const Person& p){ return p.age < 18; });
```

Когда нужно запомнить состояние (сделать поле данных у функтора):

```c++
auto find_first_younger(const std::vector<Person>& people, int age)
{
    return std::find_if(people.begin(), people.end(),
                        [](const Person& p){ return p.age < age; });    
}

// Не скомпилируется, т.к. компилятор не знает, каким образом
// хранить age в лямбде. Ему нужно это явно указать.
```

Захватывать данные в лямбду можно двумя способами:
1. с копированием значения
    * можно представлять как отдельное поле типа `T` в функторе
2. по ссылке
    * можно представлять  как отдельное поле типа `T*` или `T&` в функторе

```c++
int age = 33;
auto too_young = [age](const Person& p){ return p.age < age; }; 
age = 18;
// age будет скопирован по значению
// в поиске используется значение 33

std::find_if(people.begin(), people.end(), too_young);
```

```c++
int age = 33;
auto too_young = [&age](const Person& p){ return p.age < age; }; 
age = 18;
// age будет использован по ссылке
// в поиске используется значение 18

std::find_if(people.begin(), people.end(), too_young);
```

Можно указывать способ захвата "по умолчанию". Тогда все внешние объекты, упоминаемые внутри лямбды, будут захвачены соответственно:

* всё захватывать по значению:

```c++
int age = 33;
auto too_young = [=](const Person& p){ return p.age < age; };
```

* всё захватывать по ссылке:

```c++
int age = 33;
auto too_young = [&](const Person& p){ return p.age < age; };
```

Примеры:

```c++
// объект int очень дешёв для копирования,
// а разыменовывать по ссылке его дольше,
// поэтому захватим по значению

int age = read_age_limit();

auto it = std::find_if(people.begin(), people.end(),
                       [age](const Person& p) { return p.age < age; });
```

```c++
// объект std::set<std::string> очень дорогой для копирования,
// лучше потратить немножко на дополнительные jump-ы по ссылкам,
// поэтому захватим по ссылке

std::set<std::string> allowed_names = ...;
                
auto it = std::find_if(people.begin(), people.end(),
                       [&allowed_names](const Person& p) { return allowed_names.count(p.name) > 0; });
```

**Упражнение:** что здесь происходит?

```c++
std::set<std::string> allowed_names = ...;
std::set<std::string>* p_allowed_names = &allowed_names;
                
auto it = std::find_if(people.begin(), people.end(),
                       [p_allowed_names](const Person& p) { return p_allowed_names->count(p.name) > 0; });
```

<br />

##### Dangling references problem

```c++
auto make_names_checker()
{
    std::set<std::string> allowed_names = ...;                
    return [&allowed_names](const Person& p) { return allowed_names.count(p.name) > 0; };
}

auto names_checker = make_names_checker();
names_checker(Person("Ilya"));  // ooooops
```

* какая здесь проблема?
* как с ней бороться?
* правила?

<br />

##### Возвращаемый тип

Компилятор автоматически выводит возвращаемый тип, опираясь на тип выражения в `return`. Но можно и указать тип явно:

```c++
auto too_young = [](const Person& p) { return p.age < 18; };
```

```c++
auto too_young = [](const Person& p) -> bool { return p.age < 18; }
```

**Упражнение**: в каких случаях это необходимо?

Ответ-пример:

```c++
// без явного указания типа возвращемого значения код не скопилируется
auto build_full_name = [](const Person& p) -> std::string {

    // return type is const char[9]
    if (p.name.empty() && p.surname.empty())
        return "unknown";

    // return type is std::string
    return p.name + " " + p.surname;
};
```

```c++
// без явного указания типа возвращемого значения код не скопилируется
auto find_lowest_point = [](const std::vector<Point>& points) -> std::optional<Point> {
 
    // return type is std::nullopt_t
    if (points.empty())
        return std::nullopt;
    
    // return type is Point
    return *std::min_element(points.begin(), points.end(),
                             [](const Point l, const Point r) { return l.z < r.z; });
};
```

<br />

##### mutable - лямбды

Использование ключевого слова `mutable` означает, что данные, захваченные копированием значения, лямбда вправе поменять:

```c++
void mutable_lambda_example()
{
    auto fun = [i = 0]() mutable -> bool {
        ++i;
        return i != 4;
    };
    while (fun())
        std::cout << "ho";  // напечатает hohoho
}
```

<br />

##### immediately call lambdas

Рассмотрим пример:

```c++
void process_log_file(const std::string& basename)
{
    const std::string filepath = basename + ".log";
    std::ifstream ifs(filepath);
    if (!ifs)
        throw std::runtime_error("filed to open log file on read");

    std::vector<Record> log_records{std::istreambuf_iterator<Record>(ifs),
                                    std::istreambuf_iterator<Record>()};
    std::reverse(log_records.begin(), log_records.end());
    
    
    ... 200+ lines of records processing further
}
```

Проблемы такой реализации:
* скорее всего `filepath` и `ifs` больше не нужны, но:
    - переменные видны в коде ниже и надо разбираться, нужны они там или нет
    - переменные держат за собой память и открытый дескриптор файла, а эти ресурсы уже можно и освободить
* формирование `log_records` завершено и далее меняться не будет. Его бы как-нибудь пометить `const`, но язык такое не позволяет.

Возможные решения: выделить функцию `read_log_records` или завернуть в immediately call lambda:

```c++
void process_log_file(const std::string& basename)
{
    const std::vector<Record> log_records = [&basename](){
        const std::string filepath = basename + ".log";
        std::ifstream ifs(filepath);
        if (!ifs)
            throw std::runtime_error("filed to open log file on read");

        std::vector<Record> log_records{std::istreambuf_iterator<Record>(ifs),
                                        std::istreambuf_iterator<Record>()};
        std::reverse(log_records.begin(), log_records.end());
        return log_records;
    }();
    
    ... 200+ lines of records processing further
}
```

Не стоит бояться оверхеда в релизе на вызов immediately call - лямбд. Компиляторы хороши в их оптимизации:

(закинуть код на godbolt.org)

```c++
// оптимизация для compile-time вычислений
int get_5() noexcept {
    return [](){ return 5; }();    
}

// оптимизация для чуть более сложных функций
double f_1(const double x, const double y) noexcept {
    const double l = x*x +  y*y;
    return x / l + y / l;
}

double f_2(const double x, const double y) noexcept {
    const double add1 = [=](){
        const double l = x*x + y*y;
        return x / l;
    }();
    const double add2 = [=](){
        const double l = x*x + y*y;
        return y / l;
    }();
    return add1 + add2;
}
```

Конечно, пример не есть доказательство, но, тем не менее, и gcc, и clang смогли сгенерировать одинаковый код для `f_1` и `f_2`, несмотря на наличие лишних вызовов и доп. вычислений в `f_2`.

На уровне оптимизаций для отладки `-O0` компилятор не может себе позволить инлайнить immediately call лямбды, там код отличается, это нормально.

Поэтому, если performance для `-O0` важен (бывают такие проекты), то в горячих функциях (коих **очень мало**) лучше воздержаться от immediately call.

<br />

##### generic lambda

Ничто не мешает сделать `operator()` шаблонным:

```c++
struct HTMLPrinter
{
    template<typename T>
    void operator()(const T& x) const { std::cout << "<p> value = " << x << " </p>"; }
};
```

Аналогично можно поступить и для лямбды, заменив тип аргумента словом `auto`

```c++
auto too_young = [](const auto& p) { return p.age < 18; };
```

generic lambda как и template-метод позволяют вызывать для разных аргументов

```c++
bool f()
{
    auto less = [](const auto& x, const auto& y) { return x < y; };
    return less(3, 5) && less(3.0, 5.0);
}
```

Пару слов об использовании и цене

<br />

##### `noexcept` lambda

`noexcept` имеет аналогичную семантику как и `noexcept` для обычных функций.

```c++
auto too_young = [](const auto& p) noexcept { return p.age < 18; };
```

<br />

##### Сохранение лямбда-функции как объекта.

Проблема в записи лямбды в поле класса заключается в том, что тип лямбда-функции создаётся в момент генерации лямбды. Его нельзя выписать явно (иногда можно неявно).

Варианты решения проблемы:
1. Использовать шаблоны
2. Заворачивать лямбды в `std::function`

```c++
class ImagesManager {
public:
    ImagesManager(std::function<Image(int)> img_searcher)
        : img_searcher_(std::move(img_searcher))
    {}
    
    ...    
private:
    std::function<Image(int)> img_searcher_;
};

// client code:
auto search_image = [](const int key) -> Image { ... };
ImagesManager manager(search_image);
```

```c++
template<typename ImgSearcherT>
class ImagesManager {
public:
    ImagesManager() {}
    
    ...    
private:
    ImgSearcherT img_searcher_;
};


// объяснить, почему это вообще работает
// client code:
auto search_image = [](const int key) -> Image { ... };
ImagesManager<decltype(search_image)> manager;
```

**Упражнение**: каковы плюсы и минусы обоих из вариантов?